In [ ]:
!pip install tensorflow
!pip install matplotlib

#### Step 1: Loading dataset and preproccessing

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import math

2023-08-21 14:22:50.879427: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 14:22:50.914134: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 14:22:50.914757: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 14:22:51.708727: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

def one_hot_encoder(class_number,label_size):
    label = np.zeros(label_size)
    label[class_number] = 1
    return label

x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

x_train.reshape(x_train.shape[0],32,32,3)
x_test.reshape(x_test.shape[0],32,32,3)

y_train = np.array([one_hot_encoder(class_number,10) for class_number in y_train])
y_test = np.array([one_hot_encoder(class_number,10) for class_number in y_test])

#### Step 2: Train Model, save checkpoints and stopping.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout

In [4]:
checkpoint_filepath = 'ckpt/checkpoint-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_filepath)

batch_size = 32
n_batches = len(x_train) / batch_size
n_batches = math.ceil(n_batches) 

model_checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_filepath,
    verbose=1,
    save_weights_only=True,
    save_freq=2*n_batches)

In [5]:
def create_model():
    model = Sequential([
    Conv2D(16, kernel_size=(3, 3), activation=tf.keras.activations.relu, input_shape=(32,32,3)), # 30x30x16
    Conv2D(32, kernel_size=(3, 3), activation=tf.keras.activations.relu), # 28x28x32
    MaxPooling2D(pool_size=(2, 2)), # 14x14x32
    Conv2D(64, (3, 3), activation=tf.keras.activations.relu), # 14x14x64
    MaxPooling2D(pool_size=(2, 2)), # 7x7x64
    Flatten(),
    Dense(64, activation=tf.keras.activations.relu),
    Dense(32, activation=tf.keras.activations.relu),
    Dense(10, activation=tf.keras.activations.softmax)])

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(0.01), metrics=tf.keras.metrics.CategoricalAccuracy())

    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0

2023-08-21 14:22:58.991587: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 14:22:58.991943: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 173514 (677.79 KB)
Trainable params: 173514 (677.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.save_weights(checkpoint_filepath.format(epoch=0))

In [7]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=20, validation_data=(x_test,y_test), callbacks=[model_checkpoint_callback])

Epoch 1/20


2023-08-21 14:23:42.251281: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 29s 18ms/step - loss: 2.1341 - categorical_accuracy: 0.2096 - val_loss: 1.8625 - val_categorical_accuracy: 0.3211
Epoch 2/20
1562/1563 [============================>.] - ETA: 0s - loss: 1.6948 - categorical_accuracy: 0.3897
Epoch 2: saving model to ckpt/checkpoint-0002.ckpt
1563/1563 [==============================] - 29s 18ms/step - loss: 1.6948 - categorical_accuracy: 0.3897 - val_loss: 1.5330 - val_categorical_accuracy: 0.4507
Epoch 3/20
1563/1563 [==============================] - 28s 18ms/step - loss: 1.4521 - categorical_accuracy: 0.4762 - val_loss: 1.4238 - val_categorical_accuracy: 0.4937
Epoch 4/20
1562/1563 [============================>.] - ETA: 0s - loss: 1.3299 - categorical_accuracy: 0.5264
Epoch 4: saving model to ckpt/checkpoint-0004.ckpt
1563/1563 [==============================] - 30s 19ms/step - loss: 1.3298 - categorical_accuracy: 0.5264 - val_loss: 1.3190 - val_categorical_accuracy: 0.5316
Epoch 5/20
1563/1563 [=========

KeyboardInterrupt: 

In [8]:
os.listdir(checkpoint_dir)

['checkpoint-0014.ckpt.index',
 'checkpoint-0006.ckpt.data-00000-of-00001',
 'checkpoint',
 'checkpoint-0010.ckpt.data-00000-of-00001',
 'checkpoint-0000.ckpt.index',
 'checkpoint-0000.ckpt.data-00000-of-00001',
 'checkpoint-0016.ckpt.index',
 'checkpoint-0004.ckpt.data-00000-of-00001',
 'checkpoint-0012.ckpt.index',
 'checkpoint-0008.ckpt.index',
 'checkpoint-0010.ckpt.index',
 'checkpoint-0006.ckpt.index',
 'checkpoint-0012.ckpt.data-00000-of-00001',
 'checkpoint-0016.ckpt.data-00000-of-00001',
 'checkpoint-0002.ckpt.data-00000-of-00001',
 'checkpoint-0004.ckpt.index',
 'checkpoint-0014.ckpt.data-00000-of-00001',
 'checkpoint-0002.ckpt.index',
 'checkpoint-0008.ckpt.data-00000-of-00001']

#### Step 3: Loading last checkpoint, retrain model with last checkpoint and evaluate

In [9]:
ckpt = tf.train.latest_checkpoint(checkpoint_dir)

In [10]:
model.load_weights(ckpt)

In [11]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_data=(x_test,y_test))

Epoch 1/10
   8/1563 [..............................] - ETA: 27s - loss: 0.5070 - categorical_accuracy: 0.8086

2023-08-21 14:34:16.977205: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 25s 16ms/step - loss: 0.6087 - categorical_accuracy: 0.7862 - val_loss: 1.0167 - val_categorical_accuracy: 0.6812
Epoch 2/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.5736 - categorical_accuracy: 0.7985 - val_loss: 1.0057 - val_categorical_accuracy: 0.6744
Epoch 3/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.5423 - categorical_accuracy: 0.8111 - val_loss: 0.9779 - val_categorical_accuracy: 0.6888
Epoch 4/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.5100 - categorical_accuracy: 0.8210 - val_loss: 1.0033 - val_categorical_accuracy: 0.6916
Epoch 5/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.4803 - categorical_accuracy: 0.8294 - val_loss: 1.0091 - val_categorical_accuracy: 0.6889
Epoch 6/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.4483 - categorical_accuracy: 0.8420 - val_loss: 1.0318 - val_categorical_accuracy

In [12]:
model.save('model.h5')

/home/mrfoziljon/checkpoint/env/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
